# Tutorial 4: Ocean wind driven circulation

**Week 1, Day 2: State of the Climate - Ocean and Atmospheric Reanalysis**
**By Neuromatch Academy**

__Content creators:__   Abigail Bodner,   Day Lead's Name

__Content reviewers:__  Ohad Zivan

__Content editors:__   Jenna Pearson, Brodie Pearson, Ohad Zivan and Chi Zhang

__Production editors:__ Name Surname, Name Surname

**Our 2023 Sponsors:** NASA TOPS & Neuromatch

### Tutorial Objectives

The ocean's motion is driven by radiation from the sun, winds, and various sources of fresh water  (precipitation, rivers, melting and freezing ice). This tutorial focuses on global atmospheric wind patterns and how these drive the "*wind driven circulation*" component of large-scale ocean currents. The wind driven circulation is important for coastal communities and climate. It impacts the ocean temperature and upwelling systems along many coast lines and influences weather on many timescales. At the end of this tutorial you will be able to 

a) access and select ocean reanalysis data data 

b) plot average surface currents 

c) compare with global winds from a previous tutorial


For this tutorial we will use the *Estimating the Circulation and Climate of the Ocean (ECCO)* dataset, which makes the best possible estimates of ocean circulation and its role in climate (https://www.ecco-group.org/).

- brief slides on large-scale atmospheric dynamics (<5min slides)
-----------------------------------

In [1]:
#from intake import open_catalog
import matplotlib.pyplot as plt
import matplotlib
import ipywidgets as widgets       # interactive display
from IPython.display import IFrame
import os
import pooch

import numpy as np
import xarray as xr
import warnings
from cartopy import crs as ccrs, feature as cfeature
#  Suppress warnings issued by Cartopy when downloading data files
warnings.filterwarnings('ignore')

In [2]:
# plot helping function

def set_projection_figure(projection = ccrs.PlateCarree(), figsize =(5, 4.5) ):
    # source:https://foundations.projectpythia.org/core/cartopy/cartopy.html

    projLccNY =  projection #ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)

    fig = plt.figure(figsize=figsize)
    ax = plt.subplot(1, 1, 1, projection=projLccNY)

    format_axes(ax)
    #ax.add_feature(cfeature.STATES)
    #ax.add_feature(cfeature.RIVERS)
    return fig, ax

def format_axes(ax):
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAKES, edgecolor='black', facecolor='None', alpha=0.3 )
    gl = ax.gridlines(draw_labels=True, linewidth=1, color='black', alpha=0.5, linestyle='--')
    gl.xlocator = matplotlib.ticker.MaxNLocator(7)
    gl.ylocator = matplotlib.ticker.MaxNLocator(5)
    gl.xlabels_top  = False
    gl.ylabels_left = False
    #gl.xlines = False

# helping functions:
def geographic_lon_to_360(lon):
    return 360 + lon

def inverted_geographic_lon_to_360(lon):
    return lon - 180

def cbar_label(DD):
    return DD.attrs['long_name'] + ' [' + DD.attrs['units']+ ']'


In [3]:
# @title Figure settings
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle")


def font_for_print():

    SMALL_SIZE = 8
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 11
    legend_properties = {'weight':'bold'}

    plt.rc('font', size=SMALL_SIZE, serif='Helvetica Neue', weight='normal')          # controls default text sizes
    #plt.rc('font', size=SMALL_SIZE, serif='DejaVu Sans', weight='light')
    plt.rc('text', usetex='false')
    plt.rc('axes', titlesize=MEDIUM_SIZE, labelweight='normal')     # fontsize of the axes title
    plt.rc('axes', labelsize=SMALL_SIZE, labelweight='normal') #, family='bold')    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=SMALL_SIZE, frameon=False)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE, titleweight='bold', autolayout=True) #, family='bold')  # fontsize of the figure title

    plt.rc('axes', labelsize= SMALL_SIZE, labelweight='normal')

font_for_print()

Load atmospheric (ERA5) and oceanic (ECCO) reanalysis data 



In [9]:
#ERA5 data loading
fname_era5 = 'ERA5_select_example.nc'
url_era5 = "https://osf.io/82gba/download"
ERA5 = xr.open_dataset(pooch.retrieve(url_era5, known_hash=None, fname=fname_era5))
ERA5

<xarray.Dataset>
Dimensions:    (time: 744, latitude: 100, longitude: 101)
Coordinates:
  * latitude   (latitude) float32 55.0 54.75 54.5 54.25 ... 30.75 30.5 30.25
  * longitude  (longitude) float32 270.0 270.2 270.5 270.8 ... 294.5 294.8 295.0
  * time       (time) datetime64[ns] 2018-03-01 ... 2018-03-31T23:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    tcc        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-09-20 05:15:01 GMT by grib_to_netcdf-2.10.0: /opt/ecmw...

In [11]:
#Surface data loading
fname_surface = 'surface_eval.nc.nc'
url_surface = "https://osf.io/46e35/download"
ECCO_U = xr.open_dataarray(pooch.retrieve(url_surface, known_hash=None, fname=fname_surface))
ECCO_U

<xarray.DataArray 'EVEL' (latitude: 360, longitude: 720)>
[259200 values with dtype=float64]
Coordinates:
    i          (longitude) int64 ...
    k          int64 ...
    j          (latitude) int64 ...
  * latitude   (latitude) float64 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
    Z          float32 ...

In [12]:
fname_surface_nvel = 'surface_nvel.nc'
url_surface_nvel = "https://osf.io/v6mzq/download"
ECCO_V = xr.open_dataset(pooch.retrieve(url_surface_nvel, known_hash=None, fname=fname_surface_nvel))
ECCO_V

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 360)
Coordinates:
    i          (longitude) int64 ...
    k          int64 ...
    j          (latitude) int64 ...
  * latitude   (latitude) float64 -89.75 -89.25 -88.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float64 -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
    Z          float32 ...
Data variables:
    NVEL       (latitude, longitude) float64 ...

Examine the time ranges of each dataset

In [13]:
print('The time range is:')
print( ERA5.time[0].data.astype('M8[h]') , 'to', ERA5.time[-1].data.astype('M8[h]') )

The time range is:
2018-03-01T00 to 2018-03-31T23


List the available *time* data from a specific year (2018) to identify the *output frequency* or *temporal frequency*, which is the rate at which reanalysis data is provided.

In [16]:
ERA5.time.sel(time='2018')

<xarray.DataArray 'time' (time: 744)>
array(['2018-03-01T00:00:00.000000000', '2018-03-01T01:00:00.000000000',
       '2018-03-01T02:00:00.000000000', ..., '2018-03-31T21:00:00.000000000',
       '2018-03-31T22:00:00.000000000', '2018-03-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2018-03-01 ... 2018-03-31T23:00:00
Attributes:
    long_name:  time

Notice the array ['2000-01-01T00:00:00.000000000', '2000-01-01T01:00:00.000000000',
       '2000-01-01T02:00:00.000000000',...] consists of dates '2000-01-01' and time 'T00:00:00.000000000', which increases to 'T01:00:00.000000000'. This indicates that the ERA5 data has an *hourly* output frequency. 
       
Now let's examine the ECCO data again for the same year 2000,

In [ ]:
# ECCO.time.sel(time='2000')

Notice that the ECCO data goes from '2000-01-15T00:00:00.000000000' to '2000-02-13T00:00:00.000000000', which indicates this is *monthly* data. Note: although the timestamp lists a specific day in the middle of a month, the data are actually monthly (temporal) averages.

## Discussion
- Why do you think the atmospheric dataset have higher output frequency than the ocean dataset?
- What can we infer about the rate of change of the two systems and their role in the climate system from these output frequencies?
- What type of averaging do we need to perform on ERA5 in order to compare directly with ECCO?

## Surface currents compared with surface winds 

- Slides about wind driven circulation and introducing the ECCO model


## Section 1: Plotting global surface winds using ERA5 

Now let's plot global maps of surface winds, averaged over a month that is present in the time ranges of both ECCO and ERA5. Fortunately, we can use a similar computation to the previous tutorial!



In [ ]:
# Pick a month that overlaps between ERA5 and ECCO. Here we choose April 2000
year_month = '2018-03'

#examine the time array within this temporal selection
ERA5.sel(time=year_month).time

Next, we calculate  monthly averages from the ERA5 hourly data, so we can compare it directly to the ECCO monthly data. We then select the atmospheric variables of interest, the 10 meter wind speeds in the zonal and meridional directions.

In [ ]:
# select year and month of interest and take average over all days in month
ERA5_monthly_mean= ERA5.sel(time=year_month).mean('time')

# zonal 10m wind speed
ERA_U = ERA5_monthly_mean['u10']
# meridional 10m wind speed
ERA_V = ERA5_monthly_mean['v10']

Rather than ploting the zonal and meridional velocity components $(u,v)$ separately, we will plot the magnitude and direction of the total velocity vector. This type of plot is called a vector field. A [vector](https://glossary.ametsoc.org/wiki/Vector) is a special mathematical quantity that has both a magnitude and a direction, just like the wind! The velocity components describe how much wind is blowing in the zonal ($u$) or meridional ($v$) directions. Specifically, wind can blow in the eastward (positive $u$) or westward (negative $u$) direction, and in the northward (positive $v$) or southward (negative $v$) direction. The total velocity vector is the *vector sum* of these two components, and has variable magnitude and direction. The magnitude ($||u||)$ and direction ($\theta$) of the total velocity vector are given by

\begin{align}
||u|| = \sqrt{u^2 + v^2},  \ \  \ \ \theta = tan^{-1}(\frac{v}{u})
\end{align}

When using a computer to plot a vector field, it is usually called a quiver plot instead. We will use a [quiver function created by Ryan Abernathey](https://rabernat.github.io/intro_to_physical_oceanography/07_ekman.html) that derives the magnitude and direction of the total velocity vector from the given zonal and meridional components.

In [ ]:
# make sure to credit Ryan Abernathey here
def quiver_ERA(u, v, sampling_x=10, sampling_y=10, mag_max=None, **kwargs):

    x = u.longitude
    y = u.latitude

    # calculate magnitude of total velocity
    mag = (u**2 + v**2)**0.5

    # coarsen the grid so the arrows are distinguishable by only selecting
    # some longitudes and latitudes defined by sampling_x and sampling_y.
    slx = slice(None, None, sampling_x)
    sly = slice(None, None, sampling_y)
    sl2d = (sly, slx)

    #fig, ax = plt.subplots(**kwargs)
    fig, ax = set_projection_figure(projection = ccrs.PlateCarree(), figsize = (9, 5.5) )

    # plot contours of the magnitude
    ax.contourf(x,y,mag, vmax=mag_max, cmap='Blues_r')

    #plt.colorbar(fig, orientation='vertical', label = 'm/s', shrink= 0.55 , pad = 0.08)

    #return the quiver plot just at the locations we sampled
    return ax, ax.quiver(x[slx], y[sly], u[sl2d], v[sl2d])

In [ ]:
# plot magnitude of wind speed overlayed by quiver arrows indicating vector direction
# Note this one takes a little while

# we define contours levels that will be used in all plots the compare them better
colorlevels = np.arange(0, 10, 2)


quiver_ERA(ERA_U, ERA_V,
             mag_max=6, sampling_x=20, figsize=(16,8))
#plt.title('Date '+year_month +' | qrt(U^2 + V^2) ',loc ='left')

## Section 2: Plotting global surface currents using ECCO 

To understand how the near-surface winds impact the ocean's motion, we will examine surface currents from [ECCO (Estimating the Circulation and Climate of the Ocean)](https://www.ecco-group.org) reanalysis and compare these to the atmosphere.

First let us look at the ECCO dataset variables

In [ ]:
# ECCO

In [ ]:
year_month

We next compute the time mean in the same manner as before. We also are only interested in the surface values so we select $k=0$, which represents the upper most layer of the ocean near the surface.

In [ ]:
# ECCO_monthly_mean= ECCO.sel(time=year_month).isel(k=0)

In [ ]:
# ECCO_V= ECCO_monthly_mean['VVELMASS']
# ECCO_U= ECCO_monthly_mean['UVELMASS']

Now let us examine ECCO_U and ECCO_V 

In [ ]:
ECCO_V

In [ ]:
ECCO_U

The only dimensions now $['j', 'i_g']$ for $U$, $['j_g', 'i']$ for $V$, and $['face']$ for both. These are related to how velocity variables are computed and stored in the ECCO model. Here $'face'$ refers to the tiles of the [grid for this particular model](https://xgcm.readthedocs.io/en/latest/grid_topology.html), which are not regular.


Let's examine what the zonal ocean velocity (ECCO_u) looks like for each tile of our model

In [ ]:
# ECCO_U.plot(col='face', col_wrap=5)

Each face represents a different section within the globe. It is possible to manually "patch" these together but we will interpolate U and V to a regular lat-lon grid using a [handy package](httpshttps://ecco-v4-python-tutorial.readthedocs.io/ECCO_v4_Interpolating_Fields_to_LatLon_Grid.html://) developed specifically to make it easier to work with the ECCO model!


In [ ]:
import ecco_v4_py as ecco

In [ ]:
ECCO_V

In [ ]:
# interpolate the zonal and meridional velocities
# ECCO_U_interp = ECCO_U.interp(i_g=ECCO_U.i)
# ECCO_V_interp = ECCO_V.interp(j_g=ECCO_V.j)

# plot
# ecco.plot_proj_to_latlon_grid(ECCO.XC, ECCO.YC, 0.5*(ECCO_U_interp**2+ECCO_V_interp**2)**0.5, cmap='Blues_r', vmax=6);

Now let us add the surface winds quiver plot overlayed on top of surface surrents

In [ ]:
# make sure to credit Ryan Abernathey here
def quiver_ERA_ECCO(u, v, sampling_x=10, sampling_y=10, mag_max=None, **kwargs):
    x = u.longitude
    y = u.latitude
    mag = 0.5*(u**2 + v**2)**0.5
    slx = slice(None, None, sampling_x)
    sly = slice(None, None, sampling_y)
    sl2d = (sly, slx)
    #fig, ax = plt.subplots(**kwargs)
    #fig, ax = set_projection_figure(projection = ccrs.PlateCarree(), figsize = (9, 5.5) )
    #ax.contourf(x,y,mag, vmax=mag_max, cmap='Blues_r')
    #plt.colorbar(fig, orientation='vertical', label = 'm/s', shrink= 0.55 , pad = 0.08)
    return ax, ax.quiver(x[slx], y[sly], u[sl2d], v[sl2d])

In [ ]:
# fig = ecco.plot_proj_to_latlon_grid(ECCO.XC, ECCO.YC, 0.5*(ECCO_U_interp**2+ECCO_V_interp**2)**0.5,
#                               cmap='Blues_r', vmax=6, projection_type = 'PlateCarree');

**SOMETHING WENT WRONG-- CAN'T OVERLAY WIND QUIVER PLOT !!**

In [ ]:
# #fig, ax = set_projection_figure(projection = ccrs.PlateCarree(), figsize = (9, 5.5) )
# fig, ax = plt.subplots()
# ecco.plot_proj_to_latlon_grid(ECCO.XC, ECCO.YC, 0.5*(ECCO_U_interp**2+ECCO_V_interp**2)**0.5,
#                               cmap='Blues_r', vmax=6, projection_type = 'PlateCarree');

# sampling_x=20
# sampling_y=20
# x = ERA_U.longitude
# y = ERA_V.latitude
# slx = slice(None, None, sampling_x)
# sly = slice(None, None, sampling_y)
# sl2d = (sly, slx)

# ax.quiver(x[slx], y[sly], ERA_U[sl2d], ERA_V[sl2d])

# plt.show()

As a final step let us compare with the surface wind magnitude and direction.

#### *A few notes:*
- **Surprise fact!** The wind driven currents at the surface are not actually in the direction of the wind but rather at a ~45 degree angle. This is known as the Ekman pumping effect, for which water is either upwelled or downwelled based on the directionality of the wind. Can you notice this relationship?
- **Fun fact!** Upwelling and downwelling is actually a very important process in the El Nino phenomena we discussed earlier. The El Nino La Nina phase is identified either strong or week upwelling near the coast. .. add here story of Fisherman along the coast of Peru and why it is named El Nino.

Next activity: can you plot the coast of Peru and examine the direction of the wind and the direction of the ocean currents?

Let's make the same plot with surface temperature? What is the relationship between wind, upwelling, and temperature in this location? (make connection here to next tutorial on density)

Bonus activity! Try to repeat the steps above with different months and years. Can you identify an El Nino or La Nina event?

Zooming back out, does the wind driven circulation does not explain all of the ocean currents? What do you think is missing? In the next tutorial you will discuss density driven currents

---
# Summary




### - Winds impose a surface stress which forces ocean currents 
### - This is called the ***wind driven circulation***
